In [105]:
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from IPython.display import clear_output
from datetime import datetime

In [2]:
# Input path ke chromedriver.exe
path_chromedriver = os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe")

chrome_options = Options()
service = Service(executable_path=path_chromedriver)

In [3]:
# Open Chrome Browser
driver = webdriver.Chrome(service=service, options=chrome_options)
url = "https://www.google.com/maps"
driver.get(url)

In [15]:
def scroll_gmaps():
    scroll_bar = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]'

    scrollable_div = driver.find_element(By.XPATH,scroll_bar)
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)

    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

    while True:
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
        time.sleep(3)  # Apabila jaringan low, bisa dinaikkan angkanya
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:
            break
        last_height = new_height

In [108]:
def inititate_data():
    return {key: [] for key in ['Keyword', 'Name', 'Type', 'Rating', 'Total Reviews', 'Alamat', 'Latitude', 'Longitude', 'No Hp', 'Website', 'Plus Code', 'Last Reviewer', 'Last Review Date', 'Last Review Text']}


In [106]:
def scrapping_data(keyword):
    name = driver.find_element(By.CLASS_NAME,"DUwDvf").text

    tipe = driver.find_element(By.CLASS_NAME,"DkEaL").text
    try: #contoh case "Warung Makan Barokah"
        rating = driver.find_element(By.CSS_SELECTOR,'div.F7nice span[aria-hidden="true"]').text
        rating = float(rating.replace(",", "."))
        total_reviews = driver.find_element(By.CSS_SELECTOR,'div.F7nice').text
    
    except:
        rating = "No Rating"
        total_reviews = "No Reviews"
    Alamat = driver.find_element(By.CLASS_NAME,"Io6YTe").text

    # Latitude dan Longitude dari Link Saat ini
    import re
    current_url = driver.current_url
    match = re.search(r"@(-?\d+\.\d+),(-?\d+\.\d+)", current_url)
    if match:
        latitude = match.group(1)
        longitude = match.group(2)

    try:
        no_hp = driver.find_element(By.XPATH, "//button[@data-tooltip='Salin nomor telepon']")
        no_hp = no_hp.get_attribute("aria-label")
    except:
        no_hp = "No Phone Number"
    no_hp

    try:
        website_element = driver.find_element(By.CSS_SELECTOR, 'a.CsEnBe')
        web = website_element.get_attribute('href')
    except:
        web = "No Website"
    web

    try:
        plus_code = driver.find_element(By.XPATH, "//button[@data-tooltip='Salin Plus Codes']")
        plus_code = plus_code.get_attribute("aria-label")
    except:
        plus_code = "No Plus Code"
    plus_code


    # Cek apakah ada review atau tidak, jika ada review maka tarik datanya
    try:
        try:
            driver.find_element(By.CLASS_NAME,'w8nwRe').click()
        except:
            skip
        
        last_reviewer = driver.find_element(By.CLASS_NAME,'d4r55').text
        last_review_date = driver.find_element(By.CLASS_NAME,'rsqaWe').text
        last_review_text = driver.find_element(By.CLASS_NAME,'wiI7pd').text
    except:
        last_reviewer = "No Review"
        last_review_date = "No Review"
        last_review_text = "No Review"

    keys = ['Keyword','Name', 'Type', 'Rating', 'Total Reviews', 'Alamat', 'Latitude', 'Longitude', 'No Hp', 'Website', 'Plus Code', 'Last Reviewer', 'Last Review Date', 'Last Review Text']
    values = [keyword,name, tipe, rating, total_reviews, Alamat, latitude, longitude, no_hp, web, plus_code, last_reviewer, last_review_date, last_review_text]

    results = zip(keys, values)
    return results

In [41]:
# Kode awal, namun ada masalah beberapa lokasi di skip oleh coding
# def loop_scrapping_data(maps_data,locations):
#     index = 0
#     for data in locations:
#         try:
#             print(f"Processing index {index}")
#             data.click()
#             time.sleep(2)
#             hasil_scrapping = scrapping_data()    
#             for key,value in hasil_scrapping:
#                 maps_data[key].append(value)
#             df=pd.DataFrame(maps_data)
#             clear_output(wait=True)
#             display(df)
#             index+=1
#             print(f"Done processing index {index}")
#         except:
#             print(f"Error processing index {index}")

In [8]:
def slow_scroll():
    # Locate scrollable div
    scroll_bar_xpath = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]'
    scrollable_div = driver.find_element(By.XPATH, scroll_bar_xpath)

    # Small scroll within the div
    driver.execute_script("arguments[0].scrollTop += 150;", scrollable_div)
    time.sleep(1)  # Short wait after scroll

In [9]:
# Function baru dengan dynamic xpath, memastikan seluruh query akan diakses
def loop_scrapping_data(maps_data,locations,keyword):
    index = 3
    
    for epoch in range(0, len(locations)):
        try:
            print(f"Processing epoch {epoch}")
            base_xpath = f'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[{index}]/div/a'
            driver.find_element(By.XPATH, base_xpath).click()
            time.sleep(2) 
            
            hasil_scrapping = scrapping_data(keyword)
            
            for key, value in hasil_scrapping:
                maps_data[key].append(value)
            
            df = pd.DataFrame(maps_data)
            clear_output(wait=True)
            display(df)
            
            index += 2
            close_button = driver.find_element(By.CLASS_NAME, 'VfPpkd-icon-LgbsSe')
            close_button.click()

            slow_scroll()
            print(f"Done processing epoch {epoch}")
        except:
            print(f"Error processing epoch {epoch}")
            break

In [10]:
def count_query():
    locations = driver.find_elements(By.CLASS_NAME,"Nv2PK")
    Total_Query = len(locations)
    check_stats = driver.find_elements(By.CLASS_NAME,"DUwDvf")
    return locations,Total_Query, check_stats

In [11]:
def start_scrap(maps_data,keyword):
    locations, Total_Query, _ = count_query()
    if Total_Query>1:
        print("Update: Ditemukan multiple query, melakukan scrolling data")
        scroll_gmaps()
        locations, Total_Query, _ = count_query()
        print(f"Update: Total Query ditemukan untuk {keyword} sejumlah {Total_Query} lokasi, melakukan scrapping data")
        time.sleep(5)
        loop_scrapping_data(maps_data,locations,keyword)
    else:
        scroll_gmaps()
        print(f'Ditemukan single query untuk {keyword}, melakukan scrapping data')
        data = scrapping_data()
        for key,value in data:
            maps_data[key].append(value)
    return maps_data

In [102]:
def clean_data(df):
    # Cleaning Data
    ## Menghapus karakter yang tidak diperlukan untuk Total Reviews
    df['Total Reviews'] = df['Total Reviews'].apply(lambda x: x[5:].replace(")","") if x != "No Reviews" else x)

    ## Mengganti link website yang masih default google maps (link to create business)
    df['Website'] = df['Website'].apply(lambda x: "No Website" if x.startswith("https://business.google.com/create?") else x)

    ## Menghapus keterangan "Telepon:" pada No Hp
    df['No Hp'] = df['No Hp'].apply(lambda x: x.replace("Telepon:",""))

    ## Menghapus keterangan "Plus Codes:" pada Plus Code
    df['Plus Code'] = df['Plus Code'].apply(lambda x: x.replace("Plus Codes:",""))

    print("Update: Data Cleaning Selesai, menyimpan data ke dalam file csv")

    return df

In [104]:
def scrap_google_maps(keyword):
    # Inititate Dict Kosong
    maps_data = inititate_data()
    
    # Input Keyword ke Google Maps
    driver.get(url)
    time.sleep(3)
    driver.find_elements(By.ID, "searchboxinput")[0].send_keys(keyword)
    driver.find_elements(By.ID, "searchbox-searchbutton")[0].click()
    time.sleep(5)

    # Melakukan scrapping data
    maps_data = start_scrap(maps_data,keyword)
    
    # Melakukan cleaning data
    df = pd.DataFrame(maps_data)
    df = clean_data(df)

    return df

In [ ]:
# Input Keyword, Jika ingin lebih banyak keyword bisa diinputkan dalam list
# atau bisa pakai function scrap_google_maps() dengan memasukkan keyword
# contoh: df = scrap_google_maps("Tempat Makan")
keyword = ["Tempat Makan"] 

for key in keyword:
    df = scrap_google_maps(key)

    # Mendapatkan tanggal saat ini
    current_date = datetime.now().date()

    # Mengatur format nama file
    formatted_date = current_date.strftime("%d-%m-%Y")  # Format: DD-MM-YYYY
    formatted_time = current_date.strftime("%H-%M-%S")  # Format: HH-MM-SS
    file_name = f"data_scrap_googlemaps_{keyword}_{formatted_date}_{formatted_time}.csv"

    # Export data ke dalam file CSV
    df.to_csv(file_name, index=False)
    df.to_csv(f"{key}.csv", index=False)

In [107]:
df

,Name,Type,Rating,Total Reviews,Alamat,Latitude,Longitude,No Hp,Website,Plus Code,Last Reviewer,Last Review Date,Last Review Text
0,Rumah Makan Borobudur,Restoran Indonesia,4.2,,"Jl. Let. Jend. Suprapto No.4, Langkai, Kec. Pa...",-2.1101664,112.8141026,0812-1460-4820,No Website,"QWQ9+QV Langkai, Kota Palangka Raya, Kalimant...",ali ma'ruf,sebulan lalu,"Biasa aja tempatnya, plafonnya ada yg bocor. H..."
1,Rumah Makan Ikan Bakar Pelangi 2,Restoran Ikan Bakar,4.5,,"SAMPING UNIVERSITAS MUHAMMADIYAH, Jl. RTA Milo...",-2.1101664,112.8141026,0821-5733-7839,No Website,"QWHC+R4 Langkai, Kota Palangka Raya, Kalimant...",Indra H.,5 bulan lalu,"Pelayanan cepat, pesen ikan bakar tdk perlu nu..."
2,Rumah Makan Family,Restoran China,4.5,,"Jl. Bawean No.8, Pahandut, Kec. Jekan Raya, Ko...",-2.1101664,112.8141026,0813-7237-8881,No Website,"QWRQ+FF Pahandut, Kota Palangka Raya, Kaliman...",Mawar,seminggu lalu,Pelayan 1nya sangat buruk lihat raul wajah ny...
3,Rumah makan & warung kopi sp 4 semanggang Kali...,Rumah Makan,5.0,,"899J+CW5, Tewang Manyangen, Kec. Tewang Sangal...",-1.6814878,112.22879,No Phone Number,No Website,No Plus Code,No Review,No Review,No Review
4,Rumah Makan Cianjur,Restoran,4.3,,"QWW8+MQ6, Jl. Kahayan, Palangka, Kec. Jekan Ra...",-2.2033261,112.7633325,(0536) 3226129,No Website,No Plus Code,Dessy Diani.S,2 tahun lalu,Terendulisme memang gak ada duanya menu sate i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,RUMAH MAKAN BANG ISAR,Restoran Indonesia,4.3,,"XRF2+736, Jl. Sabrani, Parenggean, Kec. Pareng...",-2.0268477,111.6465656,No Phone Number,No Website,No Plus Code,Denni Nuari,2 tahun lalu,"Menu lumayan banyak, dan spesial hidangan samb..."
116,Warung Nasi Kak Senah,Warung nasi,4.5,,"M5P2+XJG, Telok Pa'kedai, tuan-tuan, Kabupaten...",-0.3125539,107.998028,0812-5681-8079,No Website,No Plus Code,No Review,No Review,No Review
117,Rumah Makan Batak Andaliman,Restoran Batak,4.2,,"FW7V+C4Q, Jl. Pemuda, Sawahan, Kec. Mentawa Ba...",-2.5364058,111.7892955,No Phone Number,No Website,No Plus Code,Jhosua Wahyu,2 tahun lalu,Salah satu warung langganan kalo pengen makan ...
118,RM. Rinjani,Restoran Batak,4.4,,"Jl. Rinjani No.25, Palangka, Kec. Jekan Raya, ...",-2.2027535,112.7554684,No Phone Number,No Website,"QWW5+VJ Palangka, Kota Palangka Raya, Kaliman...",Haryono sutanto,11 bulan lalu,Bersih dan ada bir👍
